In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [6]:
plt.rcParams["figure.figsize"] = (10, 6)

In [7]:
# 데이터 로드
BASE_DIR = Path("..")
DATA_DIR = BASE_DIR / "data"

VIDEO_META_PATH = DATA_DIR / "channel_videos_with_stats.csv"
df = pd.read_csv(VIDEO_META_PATH)

df.head()

,channel_label,channel_handle,video_id,video_title,published_at,view_count,like_count,comment_count,duration_sec,video_type
0,ISEGYE_IDOL,isegyeidol_ofcl,YkNTjfc45-o,(SUB) 이게 나온다고요?! 😂🎁 랜덤 선물 뽑기 - 화이트 (White) 발매 ...,2026-01-09T09:00:56Z,26203,2897,151,641.0,LONG
1,ISEGYE_IDOL,isegyeidol_ofcl,_KpApEqZp9c,이심전심 이세계아이돌 💞 #이세계아이돌 #ISEGYEIDOL #shorts,2026-01-06T09:00:29Z,125626,6939,140,48.0,SHORT
2,ISEGYE_IDOL,isegyeidol_ofcl,bZHiZn5ea90,저음불가 르르땅 🦊 과 과몰입 징부기 🍔 #이세계아이돌 #ISEGYEIDOL #sh...,2026-01-06T09:00:25Z,56493,4480,110,57.0,SHORT
3,ISEGYE_IDOL,isegyeidol_ofcl,im_YVykT91E,나만 아니면 돼애애애애!!!!!! 🤭 #이세계아이돌 #ISEGYEIDOL #shorts,2026-01-06T09:00:19Z,59328,4560,127,59.0,SHORT
4,ISEGYE_IDOL,isegyeidol_ofcl,-xKhVTH9z3s,(SUB) 게임에 진심인데요...!🔥 운명의 트리 꾸미기 🎄 - 화이트 (White...,2026-01-06T09:00:01Z,32345,3382,213,732.0,LONG


In [ ]:
# published_at을 datetime으로 변환
df["published_at"] = pd.to_datetime(df["published_at"], utc=True)

In [15]:
df["published_at"].dtype

datetime64[us, UTC]

In [8]:
df.columns

Index(['channel_label', 'channel_handle', 'video_id', 'video_title',
       'published_at', 'view_count', 'like_count', 'comment_count',
       'duration_sec', 'video_type'],
      dtype='str')

In [ ]:
# 영상 타입 분포
df["video_type"].value_counts()
# UNKOWN은 live stream

video_type
SHORT      3259
LONG       1897
UNKNOWN       1
Name: count, dtype: int64

In [10]:
# 채널별 Shorts / Long 개수
pd.crosstab(df["channel_label"], df["video_type"])

video_type,LONG,SHORT,UNKNOWN
channel_label,,,
ISEGYE_IDOL,33,56,0
IVE,656,1449,0
PLAVE,792,646,1
RIIZE,416,1108,0


ISEGYE_IDOL 채널은 2025년 9월 공식 채널 리뉴얼 이후 운영되었기 때문에,
다른 아이돌 채널과 동일 선상에서 영상 수 기준 비교를 수행할 경우
관측 기간 차이에 따른 편향이 발생할 수 있다.

이에 따라 본 분석에서는
(1) 동일 기간 기준 분석,
(2) 참여도 비율 기반 지표,
(3) 사례 분석(case study)
의 세 가지 관점을 병행하여 해석하였다.

In [17]:
cutoff = pd.Timestamp("2025-09-16", tz="UTC")

df_period = df[df["published_at"] >= cutoff]

In [18]:
pd.crosstab(
    df_period["channel_label"],
    df_period["video_type"]
)

video_type,LONG,SHORT
channel_label,,
ISEGYE_IDOL,33,56
IVE,49,189
PLAVE,49,85
RIIZE,81,175


In [ ]:
summary_mean = (
    df
    .groupby(["channel_label", "video_type"])
    .agg(
        avg_view=("view_count", "mean"),
        avg_like=("like_count", "mean"),
        avg_comment=("comment_count", "mean"),
        count=("video_id", "count")
    )
    .reset_index()
)

summary_mean

In [ ]:
summary_median = (
    df
    .groupby(["channel_label", "video_type"])
    .agg(
        med_view=("view_count", "median"),
        med_like=("like_count", "median"),
        med_comment=("comment_count", "median"),
    )
    .reset_index()
)

summary_median